In [1]:
import tnn
import torch
import torch.nn as nn
import torch.utils.data as data
import numpy as np

from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

In [2]:
dataset = load_dataset("ylecun/mnist", split="test", num_proc=2)

test_indices = np.random.choice(len(dataset), size=len(dataset), replace=False)
test = dataset.select(test_indices)

In [3]:
def to_numpy(example):
    arr = np.reshape(example["image"], -1) / 255.0
    example["input"] = arr
    return example


test_dataset = test.map(to_numpy, num_proc=2).select_columns(["input", "label"])

Map (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
def collate_fn(batch):
    inputs = torch.tensor([ex["input"] for ex in batch]).float()
    labels = torch.tensor([ex["label"] for ex in batch]).long()
    return inputs, labels

testloader = data.DataLoader(
    test_dataset, batch_size=32, shuffle=True, drop_last=False, num_workers=2, collate_fn=collate_fn,  pin_memory=True
)

In [5]:
loss_fn = nn.CrossEntropyLoss()
model = tnn.Model(tnn.MLP())
sgdm_no_nag_landscape = tnn.Landscape.from_file(
    trainer_path="../training/mnist-momentum-no-nag.h5",
    model=model,
    loss_fn=loss_fn,
    eval_dataloader=testloader,
    device=device,
    path="../landscapes/mnist-momentum-landscape-no-nag.h5",
    verbose=25
)

In [6]:
sgmd_no_nag_dict = sgdm_no_nag_landscape.create_meshgrid(resolution=25, mode="pca", endpoints=(-10, 10))

meshgrid creation using pca
model using cuda
meshgrid creation started
(iter: 25): iter loss: 0.0735
(iter: 50): iter loss: 0.0733
(iter: 75): iter loss: 0.0731
(iter: 100): iter loss: 0.0729
(iter: 125): iter loss: 0.0729
(iter: 150): iter loss: 0.0725
(iter: 175): iter loss: 0.0723
(iter: 200): iter loss: 0.0721
(iter: 225): iter loss: 0.0719
(iter: 250): iter loss: 0.0717
(iter: 275): iter loss: 0.0715
(iter: 300): iter loss: 0.0714
(iter: 325): iter loss: 0.0712
(iter: 350): iter loss: 0.0714
(iter: 375): iter loss: 0.0717
(iter: 400): iter loss: 0.0707
(iter: 425): iter loss: 0.0707
(iter: 450): iter loss: 0.0704
(iter: 475): iter loss: 0.0702
(iter: 500): iter loss: 0.0701
(iter: 525): iter loss: 0.0700
(iter: 550): iter loss: 0.0698
(iter: 575): iter loss: 0.0700
(iter: 600): iter loss: 0.0696
(iter: 625): iter loss: 0.0699
meshgrid creation complete
../landscapes
meshgrid saved to ../landscapes/mnist-momentum-landscape-no-nag.h5/landscape/meshgrid
optimizer path was saved to ..

In [7]:
sgdm_nag_landscape = tnn.Landscape.from_file(
    trainer_path="../training/mnist-momentum-nag.h5",
    model=model,
    loss_fn=loss_fn,
    eval_dataloader=testloader,
    device=device,
    path="../landscapes/mnist-momentum-landscape-nag.h5",
    verbose=25
)

In [8]:
sgdm_nag_dict = sgdm_nag_landscape.create_meshgrid(resolution=25, endpoints=(-10, 10), mode="pca")

meshgrid creation using pca
model using cuda
meshgrid creation started
(iter: 25): iter loss: 0.0753
(iter: 50): iter loss: 0.0751
(iter: 75): iter loss: 0.0749
(iter: 100): iter loss: 0.0747
(iter: 125): iter loss: 0.0747
(iter: 150): iter loss: 0.0747
(iter: 175): iter loss: 0.0745
(iter: 200): iter loss: 0.0740
(iter: 225): iter loss: 0.0738
(iter: 250): iter loss: 0.0737
(iter: 275): iter loss: 0.0748
(iter: 300): iter loss: 0.0734
(iter: 325): iter loss: 0.0732
(iter: 350): iter loss: 0.0731
(iter: 375): iter loss: 0.0729
(iter: 400): iter loss: 0.0728
(iter: 425): iter loss: 0.0726
(iter: 450): iter loss: 0.0725
(iter: 475): iter loss: 0.0723
(iter: 500): iter loss: 0.0722
(iter: 525): iter loss: 0.0721
(iter: 550): iter loss: 0.0719
(iter: 575): iter loss: 0.0718
(iter: 600): iter loss: 0.0717
(iter: 625): iter loss: 0.0717
meshgrid creation complete
../landscapes
meshgrid saved to ../landscapes/mnist-momentum-landscape-nag.h5/landscape/meshgrid
optimizer path was saved to ../la